# Predicting flight delay
Karthik Sunil, Aug 2019

-------
In this notebook, I develop a model aimed at predicting flight delays at take-off. I show how to build linear and polynomial models for univariate or multivariate regressions and also, I give some insight on the reason why regularisation helps us in developing models that generalize well.

The entire notebook is divided into 3 Major sections. 
1. Understanding and Cleaning Data
2. Re-sampling of the data
3. Exploratory Data Analysis
4. Building various model to detect the delays

In the begenning let us import all the libraries we need for this analysis purpose

In [ ]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from mpl_toolkits.basemap import Basemap
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

#### Loading the data 
In this dataset, we have 3 input files and 1 kernel output file. You can see the details in the workspace to the right

Input files:

- **flights.csv**: Contains entire dataset of the flights data for 2015. Contains around 58 million data records
- **aiports.csv**: Contains the lookup table for Airports
- **airlines.csv**: Contains the lookup table for Airlines

Kernel output file:
- **flights_date**: Contains pre-processed output file that can be readily used for data analysis. It contains the data after the clearning of date format and added lookup data to flights data. This avoids to perform costly operations again and again. However, in this kernel the cells which cleans up the data has been commented, but kept for future purposes. 

In [ ]:
airports = pd.read_csv("../input/flight-delays/airports.csv")

In [ ]:
df = pd.read_csv('../input/flight-delays/flights.csv', low_memory=False)
print('Dataframe dimensions:', df.shape)
#____________________________________________________________
# gives some infos on columns types and number of null values
tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100)
                         .T.rename(index={0:'null values (%)'}))
tab_info

___
## 1. Data Acquaintance and Cleaning
___
### 1.1 Dates and times

In the initial dataframe, dates are coded according to 4 variables: **YEAR, MONTH, DAY**, and **DAY_OF_WEEK**. In fact, python offers the **_datetime_** format which is really convenient to work with dates and times and I thus convert the dates in this format:


In [ ]:
#!ls ../input/*

**NOTE**
The cleaning for date is already done and stored in another dataset called flights_date.csv. I suggest to jump to the cell with heading *Load flight_date.csv* . The file path. [Link to cell](#load_file)

```
../input/analysis-of-flight-delay/flights_date.csv
```


In [ ]:
df['DATE'] = pd.to_datetime(df[['YEAR','MONTH', 'DAY']])

Moreover, in the **SCHEDULED_DEPARTURE** variable, the hour of the take-off is coded as a float where the two first digits indicate the hour and the two last, the minutes. This format is not convenient and I thus convert it. Finally, I merge the take-off hour with the flight date. To proceed with these transformations, I define a few functions:

In [ ]:
#_________________________________________________________
# Function that convert the 'HHMM' string to datetime.time
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure
#_____________________________________________________________________
# Function that combines a date and time to produce a datetime.datetime
def combine_date_heure(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
#_______________________________________________________________________________
# Function that combine two columns of the dataframe to create a datetime format
def create_flight_time(df, col):    
    liste = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_heure(cols))
        else:
            cols[1] = format_heure(cols[1])
            liste.append(combine_date_heure(cols))
    return pd.Series(liste)

and I call them to modify the dataframe variables:

*** 
NOTE: Following code is very expensive and takes a lot of time. Please uncomment and run if you need. Or else you can load already processed file

`../input/flights-delay-date/flights_date.csv`


In [ ]:
# df['SCHEDULED_DEPARTURE'] = create_flight_time(df, 'SCHEDULED_DEPARTURE')
# df['DEPARTURE_TIME'] = df['DEPARTURE_TIME'].apply(format_heure)
# df['SCHEDULED_ARRIVAL'] = df['SCHEDULED_ARRIVAL'].apply(format_heure)
# df['ARRIVAL_TIME'] = df['ARRIVAL_TIME'].apply(format_heure)
# #__________________________________________________________________________
# df.loc[:5, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME',
#              'ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]

**NOTE**
The above cell is very Costly Operation, hence saving the DF to a file, so that the prrocessed data can be loaded 

Please run the following code to save the file to local and upload back to kernel for future usage if needed. As of now the file is processed and loaded

`../input/flights-delay-date/flights_date.csv`



In [ ]:

# df.to_csv('./flights_date.csv')

# from IPython.display import FileLink
# FileLink(r'./flights_date.csv')

#### End of Data Processing

<a id='load_file'> </a>
#### LOAD flight_date.csv
As the date conversion and other pre-processing steps are expensive for 5.8 million records, I am loading already pre-processed file

In [ ]:
df = pd.read_csv('../input/flights-delay-date/flights_date.csv', low_memory=False)
df['SCHEDULED_DEPARTURE']= pd.to_datetime(df['SCHEDULED_DEPARTURE']) 

This is how the data looks like. 

In [ ]:
df.head()

### 1.2 Analysis of cancelled flights
As we are analysing the delays, we dont need to consider the flights those have been cancelled. In this section, let us see how many such flights are there and remove them

In [ ]:
total_count = df['AIRLINE'].count()
cancelled_count = df[df['CANCELLED'] == 1]['AIRLINE'].count()

print('Total flights', total_count, 'and Cancelled flights',cancelled_count, '. So % of cancelled flights: ', round(cancelled_count/total_count*100,2) ,'%')

Let us remove those cancelled flights, as they will not add any value in performing prediction of flight delays

In [ ]:
df = df[df['CANCELLED'] == 0]

Note that in practice, the content of the **DEPARTURE_TIME** and **ARRIVAL_TIME** variables can be a bit misleading since they don't contain the dates. For exemple, in the first entry of the dataframe, the scheduled departure is at 0h05 the 1st of January. The **DEPARTURE_TIME** variable indicates 23h54 and we thus don't know if the flight leaved before time or if there was a large delay. Hence, the **DEPARTURE_DELAY** and **ARRIVAL_DELAY** variables proves more useful since they directly provides the delays in minutes. Hence, in what follows, I will not use the **DEPARTURE_TIME** and **ARRIVAL_TIME** variables.

### 1.3 Filling factor

Finally, I clean the dataframe throwing the variables I won't use and re-organize the columns to ease its reading:

In [ ]:
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF','DATE', 'AIR_SYSTEM_DELAY',
                       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'AIR_TIME']
df.drop(variables_to_remove, axis = 1, inplace = True)
df = df[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
        'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
        'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
        'SCHEDULED_TIME',  'DAY_OF_WEEK', 'ELAPSED_TIME', 'MONTH']]
df[:5]

At this stage, I examine how complete the dataset is:

In [ ]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

In [ ]:
df.shape

We see that the variables filling factor is quite good (> 99%). Since we have pretty good number of samples, I decide to proceed without trying to impute what's missing and I simply remove the entries that contain missing values. The shape of our dataset is: 

In [ ]:
df.dropna(inplace = True)
df.shape

### 1.4 Perform resampling for analysis
We can see that the size of the dataset is 5.7 million records and any operations on such a huge dataset will be very expensive. So, we see  if we can re-sample the dataset, for analysis purposes. 

As we know this dataset it for entire 2015 year, let us see if we can take few months data. 
Let us see if the MONTH feature affects the delay by large way or not

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax = sns.barplot(data=df, x='MONTH',y='DEPARTURE_DELAY', ax=ax, ci=None)

From above chart, we see that MONTH feature affects the delay time. We see Jun is most affected month, followed by Dec, Feb and Jul. We also see that Sep and Oct months witnessed least delay. 

We can take one month from each Quarter for analysis purpose. Let us consider follwoing months - Feb, Jun, Aug and Dec. This will help in saving time during EDA and also modelling. 

In [ ]:
# df_original contains data with all 12 months for later analysis
# df_original = df
df = df[df['MONTH'].apply(lambda x:x in [2,6,8,12])  ]
df.shape

___
## 2. Explortory Data Analysis
In this section we perform various steps to understand the data. We will derive some intuitions which will help us in preparing the model


### 2.1 Comparing airlines
Let us compare different Airlines. As said earlier, the **AIRLINE** variable contains the airline abreviations. Their full names can be retrieved from the `airlines.csv` file.


In [ ]:
airlines_names = pd.read_csv('../input/flight-delays/airlines.csv')
airlines_names

For further use, I put the content of this this dataframe in a dictionary:

In [ ]:
abbr_companies = airlines_names.set_index('IATA_CODE')['AIRLINE'].to_dict()

___
### 2.2 Basic statistical description of airlines

As a first step, I consider all the flights from all carriers. Here, the aim is to classify the airlines with respect to their punctuality and for that purpose, I compute a few basic statisticial parameters:

In [ ]:
#__________________________________________________________________
# function that extract statistical parameters from a grouby objet:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
#_______________________________________________________________
# Creation of a dataframe with statitical infos on each airline:
global_stats = df['DEPARTURE_DELAY'].groupby(df['AIRLINE']).apply(get_stats).unstack()
global_stats = global_stats.sort_values('count')
global_stats

Now, in order to understand the data, let us see some graphics:

In [ ]:
font = {'family' : 'normal', 'weight' : 'bold', 'size'   : 15}
mpl.rc('font', **font)
import matplotlib.patches as mpatches
#__________________________________________________________________
# I extract a subset of columns and redefine the airlines labeling 
df2 = df.loc[:, ['AIRLINE', 'DEPARTURE_DELAY']]
df2['AIRLINE'] = df2['AIRLINE'].replace(abbr_companies)
#________________________________________________________________________
colors = ['royalblue', 'grey', 'wheat', 'c', 'firebrick', 'seagreen', 'lightskyblue',
          'lightcoral', 'yellowgreen', 'gold', 'tomato', 'violet', 'aquamarine', 'chartreuse']
#___________________________________
fig = plt.figure(1, figsize=(16,15))
gs=GridSpec(2,2)             
ax1=fig.add_subplot(gs[0,0]) 
ax2=fig.add_subplot(gs[0,1]) 
ax3=fig.add_subplot(gs[1,:]) 
#------------------------------
# Pie chart n0. 1: nb of flights
#------------------------------
labels = [s for s in  global_stats.index]
sizes  = global_stats['count'].values
explode = [0.3 if sizes[i] < 20000 else 0.0 for i in range(len(abbr_companies))]
patches, texts, autotexts = ax1.pie(sizes, explode = explode,
                                labels=labels, colors = colors,  autopct='%1.0f%%',
                                shadow=False, startangle=0)
for i in range(len(abbr_companies)): 
    texts[i].set_fontsize(14)
ax1.axis('equal')
ax1.set_title('% of flights per company', bbox={'facecolor':'midnightblue', 'pad':5},
              color = 'w',fontsize=18)
#_______________________________________________
# I set the legend: abreviation -> airline name
comp_handler = []
for i in range(len(abbr_companies)):
    comp_handler.append(mpatches.Patch(color=colors[i],
            label = global_stats.index[i] + ': ' + abbr_companies[global_stats.index[i]]))
ax1.legend(handles=comp_handler, bbox_to_anchor=(0.2, 0.9), 
           fontsize = 13, bbox_transform=plt.gcf().transFigure)
#----------------------------------------
# Pie chart nº2: mean delay at departure
#----------------------------------------
sizes  = global_stats['mean'].values
sizes  = [max(s,0) for s in sizes]
explode = [0.0 if sizes[i] < 20000 else 0.01 for i in range(len(abbr_companies))]
patches, texts, autotexts = ax2.pie(sizes, explode = explode, labels = labels,
                                colors = colors, shadow=False, startangle=0,
                                autopct = lambda p :  '{:.0f}'.format(p * sum(sizes) / 100))
for i in range(len(abbr_companies)): 
    texts[i].set_fontsize(14)
ax2.axis('equal')
ax2.set_title('Mean delay at origin', bbox={'facecolor':'midnightblue', 'pad':5},
              color='w', fontsize=18)
#------------------------------------------------------
# striplot with all the values reported for the delays
#___________________________________________________________________
# I redefine the colors for correspondance with the pie charts
colors = ['firebrick', 'gold', 'lightcoral', 'aquamarine', 'c', 'yellowgreen', 'grey',
          'seagreen', 'tomato', 'violet', 'wheat', 'chartreuse', 'lightskyblue', 'royalblue']
#___________________________________________________________________
ax3 = sns.stripplot(y="AIRLINE", x="DEPARTURE_DELAY", size = 4, palette = colors,
                    data=df2, linewidth = 0.5,  jitter=True)
plt.setp(ax3.get_xticklabels(), fontsize=14)
plt.setp(ax3.get_yticklabels(), fontsize=14)
ax3.set_xticklabels(['{:2.0f}h{:2.0f}m'.format(*[int(y) for y in divmod(x,60)])
                         for x in ax3.get_xticks()])
plt.xlabel('Departure delay', fontsize=18, bbox={'facecolor':'midnightblue', 'pad':5},
           color='w', labelpad=20)
ax3.yaxis.label.set_visible(False)
#________________________
plt.tight_layout(w_pad=3) 

Considering the first pie chart that gives the percentage of flights per airline, we see that there is some disparity between the carriers. For exemple, *Southwest Airlines* accounts for $\sim$20% of the flights which is similar to the number of flights chartered by the 7 tiniest airlines. However, if we have a look at the second pie chart, we see that here, on the contrary, the differences among airlines are less pronounced. Excluding *Hawaiian Airlines* and *Alaska Airlines* that report extremely low mean delays, we obtain that a value of **$\sim$11$\pm$7 minutes** would correctly represent all mean delays. Note that this value is quite low which mean that the standard for every airline is to respect the schedule !

Finally, the figure at the bottom makes a census of all the delays that were measured in January 2015. This representation gives a feeling on the dispersion of data and put in perspective the relative homogeneity that appeared in the second pie chart. Indeed, we see that while all mean delays are around 10 minutes, this low value is a consequence of the fact that a majority of flights take off on time. However, we see that occasionally, we can face really large delays that can reach a few tens of hours !

---
### 2.3 Delays Categorization





The two numbers match up, which infers that only the flights with arrival delay >= 15 minutes having a detailed delay breakdown (e.g. air system, airline, weather).

So, let us try to categorize the delays as:
- Anything <15 min as **On time**
- 15 min to 1 hour as **Short Delay**
- Beyond 1 hour as **Long Delay** 

The different type of delays are visible in the next figure:

In [ ]:
#_____________________________________________
# Function that define how delays are grouped
delay_type = lambda x:((0,1)[x > 15],2)[x > 60]
df['DELAY_LEVEL'] = df['DEPARTURE_DELAY'].apply(delay_type)
#____________________________________________________
fig = plt.figure(1, figsize=(10,7))
ax = sns.countplot(y="AIRLINE", hue='DELAY_LEVEL', data=df)
#____________________________________________________________________________________
# We replace the abbreviations by the full names of the companies and set the labels
labels = [abbr_companies[item.get_text()] for item in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.setp(ax.get_xticklabels(), fontsize=12, weight = 'normal', rotation = 0);
plt.setp(ax.get_yticklabels(), fontsize=12, weight = 'bold', rotation = 0);
ax.yaxis.label.set_visible(False)
plt.xlabel('Flight count', fontsize=16, weight = 'bold', labelpad=10)
#________________
# Set the legend
L = plt.legend()
L.get_texts()[0].set_text('on time (t < 15 min)')
L.get_texts()[1].set_text('small delay (15 < t < 60 min)')
L.get_texts()[2].set_text('large delay (t > 60 min)')
plt.show()

This figure gives a count of the delays of less than 15 minutes, those in the range 15 < t < 60 min and finally, the delays greater than 60 minutes. Hence, we see that independently of the airline, delays greater than 60 minutes only account for a few percents. However, the proportion of delays in these three groups depends on the airline: as an exemple, in the case of *SkyWest Airlines*, the flights with delay greater than 60 minutes are more than $\sim$50% with respect to delays in the range 15 < t < 60 min. Things are better for *SoutWest Airlines*  since delays greater than 60 minutes are around $\sim$35% of that of in the range 15 < t < 60 min.


In [ ]:
total = df['AIRLINE'].count()
ontime = df[df['DELAY_LEVEL'] == 0]['AIRLINE'].count()
pct_ontime = ontime/total*100
print('Ontime%: ',pct_ontime)

small_delay = df[df['DELAY_LEVEL'] == 1]['AIRLINE'].count()
pct_small_delay = small_delay/total*100
print('Small delay%: ', pct_small_delay)


large = df[df['DELAY_LEVEL'] == 2]['AIRLINE'].count()
pct_large = large/total*100
print('Large delay%: ', pct_large)

---
### 2.4 Delays distribution: establishing the ranking of airlines

It was shown in the previous section that mean delays behave homogeneously among airlines (apart from two extrem cases) and is around 11$\pm$7 minutes. Then, we saw that this low value is a consequence of the large proportion of flights that take off on time. However, occasionally, large delays can be registred. In this section, I examine more in details the distribution of delays for every airlines:

In [ ]:
#___________________________________________
# Model function used to fit the histograms
def func(x, a, b):
    return a * np.exp(-x/b)
#-------------------------------------------
points = [] ; label_company = []
fig = plt.figure(1, figsize=(11,11))
i = 0
for carrier_name in [abbr_companies[x] for x in global_stats.index]:
    i += 1
    ax = fig.add_subplot(5,3,i)    
    #_________________________
    # Fit of the distribution
    n, bins, patches = plt.hist(x = df2[df2['AIRLINE']==carrier_name]['DEPARTURE_DELAY'],
                                range = (15,180), normed=True, bins= 60)
    bin_centers = bins[:-1] + 0.5 * (bins[1:] - bins[:-1])    
    popt, pcov = curve_fit(func, bin_centers, n, p0 = [1, 2])
    #___________________________
    # bookeeping of the results
    points.append(popt)
    label_company.append(carrier_name)
    #______________________
    # draw the fit curve
    plt.plot(bin_centers, func(bin_centers, *popt), 'r-', linewidth=3)    
    #_____________________________________
    # define tick labels for each subplot
    if i < 10:
        ax.set_xticklabels(['' for x in ax.get_xticks()])
    else:
        ax.set_xticklabels(['{:2.0f}h{:2.0f}m'.format(*[int(y) for y in divmod(x,60)])
                            for x in ax.get_xticks()])
    #______________
    # subplot title
    plt.title(carrier_name, fontsize = 14, fontweight = 'bold', color = 'darkblue')
    #____________
    # axes labels 
    if i == 4:
        ax.text(-0.3,0.9,'Normalized count of flights', fontsize=16, rotation=90,
            color='k', horizontalalignment='center', transform = ax.transAxes)
    if i == 14:
        ax.text( 0.5, -0.5 ,'Delay at origin', fontsize=16, rotation=0,
            color='k', horizontalalignment='center', transform = ax.transAxes)
    #___________________________________________
    # Legend: values of the a and b coefficients
    ax.text(0.68, 0.7, 'a = {}\nb = {}'.format(round(popt[0],2), round(popt[1],1)),
            style='italic', transform=ax.transAxes, fontsize = 12, family='fantasy',
            bbox={'facecolor':'tomato', 'alpha':0.8, 'pad':5})
    
plt.tight_layout()

This figure shows the normalised distribution of delays that I modelised with an exponential distribution $ f(x) = a \, \mathrm{exp} (-x/b)$. The $a$ and $b$ parameters obtained to describe each airline are given in the upper right corner of each panel. Note that the normalisation of the distribution implies that $\int f(x) \, dx \sim 1$. Here, we do not have a strict equality since the normalisation applies the histograms but not to the model function. However, this relation entails that the $a$ and $b$ coefficients will be correlated with $a \propto 1/b$ and hence, only one of these two values is necessary to describe the distributions. Finally, according to the value of either $a$ or $b$, it is possible to establish a ranking of the companies: the low values of $a$ will correspond to airlines with a large proportion of important delays and, on the contrary, airlines that shine from their punctuality will admit hight $a$ values:

Based on above explanation, we can decide that Hawaiin Airlines is most punctual followed by Alaska, Southwest and Delta Airlines. But, we can recall here that Hawaiin and Alaska Airliens have very less volume of flights.

___
### 2.5 Delays: take-off or landing ?
In the previous section, all the discussion was done on departure delays. However, these delays differ somewhat from the delays recorded at arrival:

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['hatch.linewidth'] = 2.0  

fig = plt.figure(1, figsize=(11,6))
ax = sns.barplot(x="DEPARTURE_DELAY", y="AIRLINE", data=df, color="lightskyblue", ci=None)
ax = sns.barplot(x="ARRIVAL_DELAY", y="AIRLINE", data=df, color="r", hatch = '///',
                 alpha = 0.0, ci=None)
labels = [abbr_companies[item.get_text()] for item in ax.get_yticklabels()]
ax.set_yticklabels(labels)
ax.yaxis.label.set_visible(False)
plt.xlabel('Mean delay [min] (@departure: blue, @arrival: hatch lines)',
           fontsize=14, weight = 'bold', labelpad=10);

On this figure, we can see that delays at arrival are generally lower than at departure. This indicates that airlines adjust their flight speed in order to reduce the delays at arrival. In what follows, I will just consider the delays at departure but one has to keep in mind that this can differ from arrival delays.

---
### 2.6 Relationship with delays and weekday
I am hypothesising that delay can be associated with the day of the week. 

In [ ]:
ax = sns.barplot(x="DAY_OF_WEEK", y="DEPARTURE_DELAY", data=df, palette="husl", ci=None)

We can see that avg  delays are more on Mondays and Tuesday. But variation is not too much. We will not consider this as important feature that will impact delays

___
### 2.7. Relation between the origin airport and delays

I will now try to define if there is a correlation between the delays registered and the airport of origin. I recall that in the dataset, the number of airports considered is: 

In [ ]:
print("No. of airports: {}".format(len(df['ORIGIN_AIRPORT'].unique())))

In [ ]:
origin_nb = dict()
for carrier in abbr_companies.keys():
    liste_origin_airport = df[df['AIRLINE'] == carrier]['ORIGIN_AIRPORT'].unique()
    origin_nb[carrier] = len(liste_origin_airport)

In [ ]:
test_df = pd.DataFrame.from_dict(origin_nb, orient='index')
test_df.rename(columns = {0:'count'}, inplace = True)
ax = test_df.plot(kind='bar', figsize = (8,3))
labels = [abbr_companies[item.get_text()] for item in ax.get_xticklabels()]
ax.set_xticklabels(labels)
plt.ylabel('Number of airports visited', fontsize=14, weight = 'bold', labelpad=12)
plt.setp(ax.get_xticklabels(), fontsize=11, ha = 'right', rotation = 80)
ax.legend().set_visible(False)
plt.show()

___
#### 2.7.1 How the origin airport impact delays

In this section, I will have a look at the variations of the delays with respect to the origin airport and for every airline. The first step thus consists in determining the mean delays per airport:

In [ ]:
airport_mean_delays = pd.DataFrame(pd.Series(df['ORIGIN_AIRPORT'].unique()))
airport_mean_delays.set_index(0, drop = True, inplace = True)

for carrier in abbr_companies.keys():
    df1 = df[df['AIRLINE'] == carrier]
    test = df1['DEPARTURE_DELAY'].groupby(df['ORIGIN_AIRPORT']).apply(get_stats).unstack()
    airport_mean_delays[carrier] = test.loc[:, 'mean'] 

Since the number of airports is quite large, a graph showing all the information at once would be a bit messy, since it would represent around 4400 values (i.e. 312 airports $\times$ 14 airlines). Hence, I just represent a subset of the data:

In [ ]:
identify_airport = airports.set_index('IATA_CODE')['CITY'].to_dict()

In [ ]:
sns.set(context="paper")
fig = plt.figure(1, figsize=(12,12))

ax = fig.add_subplot(1,2,1)
subset = airport_mean_delays.iloc[:50,:].rename(columns = abbr_companies)
subset = subset.rename(index = identify_airport)
mask = subset.isnull()
sns.heatmap(subset, linewidths=0.01, cmap="YlGnBu", mask=mask, vmin = 0, vmax = 35)
plt.setp(ax.get_xticklabels(), fontsize=10, rotation = 85) ;
ax.yaxis.label.set_visible(False)

ax = fig.add_subplot(1,2,2)    
subset = airport_mean_delays.iloc[50:100,:].rename(columns = abbr_companies)
subset = subset.rename(index = identify_airport)
fig.text(0.5, 1.02, "Delays: impact of the origin airport", ha='center', fontsize = 18)
mask = subset.isnull()
sns.heatmap(subset, linewidths=0.01, cmap="YlGnBu", mask=mask, vmin = 0, vmax = 35)
plt.setp(ax.get_xticklabels(), fontsize=10, rotation = 85) ;
ax.yaxis.label.set_visible(False)

plt.tight_layout()

This figure allows to draw some conclusions. First, by looking at the data associated with the different airlines, we find the behavior we previously observed: for example, if we consider the right panel, it will be seen that the column associated with  *American Eagle Airlines* mostly reports large delays, while the column associated with *Delta Airlines* is mainly associated  with delays of less than 5 minutes. If we now look at the airports of origin, we will see that some airports favor late departures: see e.g. Denver, Chicago or New York. Conversely, other airports will mainly know on time departures such as Portland or Oakland.

Finally, we can deduce from these observations that there is a high variability in average delays, both between the different airports but also between the different airlines. This is important because it implies that in order to accurately model the delays, it will be necessary to adopt a model that is ** specific to the company and the home airport **. 

___
### 2.8 Temporal variability of delays

In this section, I look at the way delays vary with time. Considering the case of a specific airline and airport, delays can be easily represented by day and time 

In [ ]:
class Figure_style():
    #_________________________________________________________________
    def __init__(self, size_x = 11, size_y = 5, nrows = 1, ncols = 1):
        sns.set_style("white")
        sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 2.5})
        self.fig, axs = plt.subplots(nrows = nrows, ncols = ncols, figsize=(size_x,size_y,))
        #________________________________
        # convert self.axs to 2D array
        if nrows == 1 and ncols == 1:
            self.axs = np.reshape(axs, (1, -1))
        elif nrows == 1:
            self.axs = np.reshape(axs, (1, -1))
        elif ncols == 1:
            self.axs = np.reshape(axs, (-1, 1))
    #_____________________________
    def pos_update(self, ix, iy):
        self.ix, self.iy = ix, iy
    #_______________
    def style(self):
        self.axs[self.ix, self.iy].spines['right'].set_visible(False)
        self.axs[self.ix, self.iy].spines['top'].set_visible(False)
        self.axs[self.ix, self.iy].yaxis.grid(color='lightgray', linestyle=':')
        self.axs[self.ix, self.iy].xaxis.grid(color='lightgray', linestyle=':')
        self.axs[self.ix, self.iy].tick_params(axis='both', which='major',
                                               labelsize=10, size = 5)
    #________________________________________
    def draw_legend(self, location='upper right'):
        legend = self.axs[self.ix, self.iy].legend(loc = location, shadow=True,
                                        facecolor = 'g', frameon = True)
        legend.get_frame().set_facecolor('whitesmoke')
    #_________________________________________________________________________________
    def cust_plot(self, x, y, color='b', linestyle='-', linewidth=1, marker=None, label=''):
        if marker:
            markerfacecolor, marker, markersize = marker[:]
            self.axs[self.ix, self.iy].plot(x, y, color = color, linestyle = linestyle,
                                linewidth = linewidth, marker = marker, label = label,
                                markerfacecolor = markerfacecolor, markersize = markersize)
        else:
            self.axs[self.ix, self.iy].plot(x, y, color = color, linestyle = linestyle,
                                        linewidth = linewidth, label=label)
        self.fig.autofmt_xdate()
    #________________________________________________________________________
    def cust_plot_date(self, x, y, color='lightblue', linestyle='-',
                       linewidth=1, markeredge=False, label=''):
        markeredgewidth = 1 if markeredge else 0
        self.axs[self.ix, self.iy].plot_date(x, y, color='lightblue', markeredgecolor='grey',
                                  markeredgewidth = markeredgewidth, label=label)
    #________________________________________________________________________
    def cust_scatter(self, x, y, color = 'lightblue', markeredge = False, label=''):
        markeredgewidth = 1 if markeredge else 0
        self.axs[self.ix, self.iy].scatter(x, y, color=color,  edgecolor='grey',
                                  linewidths = markeredgewidth, label=label)    
    #___________________________________________
    def set_xlabel(self, label, fontsize = 14):
        self.axs[self.ix, self.iy].set_xlabel(label, fontsize = fontsize)
    #___________________________________________
    def set_ylabel(self, label, fontsize = 14):
        self.axs[self.ix, self.iy].set_ylabel(label, fontsize = fontsize)
    #____________________________________
    def set_xlim(self, lim_inf, lim_sup):
        self.axs[self.ix, self.iy].set_xlim([lim_inf, lim_sup])
    #____________________________________
    def set_ylim(self, lim_inf, lim_sup):
        self.axs[self.ix, self.iy].set_ylim([lim_inf, lim_sup])           

In [ ]:
#_______________________________
def func2(x, a, b, c):
    return a * x**2 +  b*x + c
#_______________________________
df['heure_depart'] =  df['SCHEDULED_DEPARTURE'].apply(lambda x:x.time())
test2 = df['DEPARTURE_DELAY'].groupby(df['heure_depart']).apply(get_stats).unstack()
fct = lambda x:x.hour*3600+x.minute*60+x.second
x_val = np.array([fct(s) for s in test2.index]) 
y_val = test2['mean']
popt, pcov = curve_fit(func2, x_val, y_val, p0 = [1, 2, 3])
test2['fit'] = pd.Series(func2(x_val, *popt), index = test2.index)

which visually gives:

In [ ]:
fig1 = Figure_style(8, 4, 1, 1)
fig1.pos_update(0, 0)
fig1.cust_plot_date(df['heure_depart'], df['DEPARTURE_DELAY'],
                    markeredge=False, label='initial data points')
fig1.cust_plot(test2.index, test2['mean'], linestyle='--', linewidth=2, label='mean')
fig1.cust_plot(test2.index, test2['fit'], color='r', linestyle='-', linewidth=3, label='fit')
fig1.style() ; fig1.draw_legend('upper left')
fig1.set_ylabel('Delay (minutes)', fontsize = 14)
fig1.set_xlabel('Departure time', fontsize = 14)
fig1.set_ylim(-15, 210)

Here, we can see that the average delay tends to increase with the departure time of day: flights leave on time in the morning  and the delay grows almost monotonously up to 30 minutes at the end of the day. In fact, this behavior is quite general and looking at other aiports or companies, we would find similar trends.

---
Let us now try to plot the avg delay by hour of the day to visualize in better way. So, using the SCHEDULED_DEPARTURE  field we create a new column called *dep_hour* which contains the hour of the day. 


In [ ]:
df['dep_hour'] = df['SCHEDULED_DEPARTURE'].apply(lambda x:x.time().hour)

Let us now see scatter plot of DEPARTURE_DELAY against dep_hour and see how temporal value is impacting the DEPARTURE_DELAY

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sns.lineplot(data=df, x='dep_hour',y='DEPARTURE_DELAY', ax=ax)

We can see that, till 5 AM, delay is more and reduces suddenly and keeps increasing till around 8PM and drops down

---
## 3. Feature Engineering
With all the data analysis done in Section 2, we can deduce that the DEPARTURE_DELAY target variable has following feature variables
1. Airline
2. Airport
3. Departure time

Out of these above features, Airline and Airport are categorical variables. In this section, we convert these categorical valriables in numerical variables using encoding techiques. We can use Label encoding or OneHot Encoding. 

In [ ]:
fct = lambda x:x.hour*3600+x.minute*60+x.second
df['heure_depart'] = df['SCHEDULED_DEPARTURE'].apply(lambda x:x.time())
df['heure_depart'] = df['heure_depart'].apply(fct)
df.head()

We encode Airline and Airport using Label Encodign 

In [ ]:
df = df[['AIRLINE','ORIGIN_AIRPORT', 'heure_depart', 'dep_hour', 'DEPARTURE_DELAY' ]]
le = LabelEncoder()
df['AIRLINE'] = le.fit_transform(df['AIRLINE'])
df['ORIGIN_AIRPORT'] = le.fit_transform(df['ORIGIN_AIRPORT'])
df.head()


We are now ready for building our models

---
## 4.0 Model Building and Testing

Due to the fact that any delay which are more than 60 min is considered as **Large Delay**, let us remove that from our original data set. We can consider them as outliers.

In [ ]:
df2 = df
df2['DEPARTURE_DELAY'] = df2['DEPARTURE_DELAY'].apply(lambda x:x if x < 60 else np.nan)
df2 = df2.dropna(how = 'any')
df2.head()

We now split the data into X and Y. X which are the features of the model are - *AIRLINE*, *ORIGIN_AIRPORT* AND departure hour (*dep_hour*). The mean of DEPARTURE_DELAY  is our Y. 

In [ ]:
features = ['AIRLINE', 'ORIGIN_AIRPORT', 'dep_hour']
df2 = df2.groupby(features ).mean()

df2 = df2.drop([ 'heure_depart'], axis=1)
df2 = df2.dropna(how='any')
df2.head()

In [ ]:
df2 = df2.reset_index()
y = df2['DEPARTURE_DELAY']
# X = df.drop(['DEPARTURE_DELAY'], axis=1)
X = df2[features]

Next we split the data into train and test data. 70% train and 30% test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### 4.1 Baseline Model

Let us now build a Naive Linear Model

In [ ]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

We use MSE and R2 as Metrics

In [ ]:
results = model.predict(X_test)
score1 = metrics.mean_squared_error(y_test,results )
score2 = metrics.r2_score(y_test,results )
print('MSE: ',score1, '  R2 Score: ', score2)

This is our baseline model. Let us try other models to beat this metric

---
### 4.2 Polynomial Model

We will try to improve the model by using polynomial model. In the section below, we create a grid search to get best parameter for Polynomial model

In [ ]:
for i in range(1,10):
    poly = PolynomialFeatures(degree = i)
    regr = linear_model.LinearRegression()
    X_ = poly.fit_transform(X_train)
    regr.fit(X_, y_train)
    X_ = poly.fit_transform(X_test)
    results = regr.predict(X_)
    score1 = metrics.mean_squared_error(y_test,results )
    score2 = metrics.r2_score(y_test,results )
    print('Model with Polynominal Degree', i, 'MSE: ',score1, '  R2 Score: ', score2)

We can see that polynomial order 4 gives best MSE result.

---
#### 4.2.1 Cross Validation for Polynomial Model

Now let use Cross Validation for Polynomial Model

In [ ]:
nb_folds = 10
for i in range(1,10):
    poly = PolynomialFeatures(degree = i)
    regr = linear_model.LinearRegression()
    X_ = poly.fit_transform(X )
    results = cross_val_predict(regr, X_, y, cv = nb_folds)
    
    score1 = np.mean(cross_val_score(regr, X_, y,cv = nb_folds, scoring = 'neg_mean_squared_error'))   
    score2 = np.mean(cross_val_score(regr, X_, y,cv = nb_folds, scoring = 'r2'))
    print('Model with Polynominal Degree', i, 'MSE: ',score1, '  R2 Score: ', score2)

---
### 4.3 Random Forest Regression
As we have seen, the data is not linear, so let us try some tree based algorithm. Let us try Random Forest Algorithm

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor(n_estimators=50, max_depth=1000 )
model2.fit(X_train, y_train)

Let us try to predict and evaluate the values 

In [ ]:
results2 = model2.predict(X_test)
score1 = metrics.mean_squared_error(y_test,results2 )
score2 = metrics.r2_score(y_test,results2 )
print('MSE: ',score1, '  R2 Score: ', score2)

We can observe that this has not improves much compared to baseline model.

---
### 4.4 Ridge Regression
Let us try to use Ridge technique, which helps to improve the model by regularization.

In [ ]:
from sklearn.linear_model import Ridge
ridgereg = Ridge(alpha=0.0001,normalize=True)
poly = PolynomialFeatures(degree = 3)
X_ = poly.fit_transform(X_train)
ridgereg.fit(X_, y_train)

In [ ]:
X_ = poly.fit_transform(X_test)
results = ridgereg.predict(X_)
score1 = metrics.mean_squared_error(y_test,results )
score2 = metrics.r2_score(y_test,results )
print('MSE: ',score1, '  R2 Score: ', score2)

Now, if we calculate the score associated to the predictions made with a regularization technique

And we can see that we obtain a reasonable score. Hence, with the current procedure, to determine the best model, we have two free parameters to adjust: the polynomial order and the $\alpha$ coefficient of the * 'Ridge Regression' *:

In [ ]:
score_min = 10000
for pol_order in range(1, 5):
    for alpha in range(0, 20, 2):
        ridgereg = Ridge(alpha = alpha/10000, normalize=True)
        poly = PolynomialFeatures(degree = pol_order)
        regr = linear_model.LinearRegression()
        X_ = poly.fit_transform(X_train)
        ridgereg.fit(X_, y_train)        
        X_ = poly.fit_transform(X_test)
        result = ridgereg.predict(X_)
        score = metrics.mean_squared_error(result, y_test)        
        if score < score_min:
            score_min = score
            parameters = [alpha/10, pol_order]
        print("n={} alpha={} , MSE = {:<0.5}".format(pol_order, alpha, score))

This grid search allows to find the best set of $\alpha$ and $n$ parameters. Let us note, however, that for this model, the estimates obtained with a linear regression or a polynomial of order 3 are quite close. Also we can see the best results are obtained for $\alpha$ value to be *0*. It means that model is ordinary least square regression model

---
### 4.5 LightGBM algorithm

**LightGBM** is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

In [ ]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.08
params['boosting_type'] = 'gbdt'
# params['boosting_type'] = 'dart'
params['objective'] = 'regression'
params['metric'] = 'mse'
params['sub_feature'] = 0.5
params['num_leaves'] = 100
params['min_data'] = 5
params['max_depth'] = 100
y_train=y_train.ravel()
reg= lgb.train(params, d_train, 100)
results=reg.predict(X_test)
score1 = metrics.mean_squared_error(y_test,results )
score2 = metrics.r2_score(y_test,results )
print('MSE: ',score1, '  R2 Score: ', score2)

We can tweak many of the LightGBM hyperparameters for better results, I observed that ```boosting_type``` and ```learning_rate``` affected the most for the results. 

## 5.0 Model Evaluation

So far LightGBM regression with ```boosting_type = gbdt``` with ```learning_rate = 0.08``` has fit the model the best way with the above MSE and R2 score.

To get an idea of the meaning of such a value for the MSE, we can assume a constant error on each point of the dataset. In which case, at each point $ i $, we have:

\begin{eqnarray}
y_i - f(x_i) = cste = \sqrt{MSE}
\end{eqnarray}

thus giving the difference in minutes between the predicted delay and the actual delay. In this case, the difference between the model and the observations is thus typically:

In [ ]:
'Estimation Error = {:.2f} min'.format(np.sqrt(score1))

## 6.0 Conclusion

By keeping the original problem which is to predict the flight departure delays, we have considered multiple features and selected top 3 features which has major impacts on delays. (Features)

- Airport
- Airline
- Time of departure (Hour)

We then constructed base model and then improvised multiple other models. At the end we found the best model based on best Metric output which is **LightGBM**. We can use that model to predict any flight delays provided Airport, Airline and time of departure. We can estimate the delay with approximation of *5~6 min*.




## Improvements
Definitely, we can improve the solution for this problem. Following are some of the improvement ideas:

* First of all, we have eliminated Cancelled flights, however in the improved model we can also predict the possibiltiy of cancellation
* We have removed all the flights which have delays more than 1 hour. We can build a 2 step soltution, first one, which classifies if the flight gets **cancelled**, **short_delay**, **large_delay**  or  **ontime**. Then in the second step, it can predict the actual delay using regression
* We can perform an ensemble of LightGBM and Ridge Regression models for better generalization 
* We can also explore more hyperparameter tuning for LightGBM to improve the model
* In this solution, we took only 4 months of data due to low power kernels, we can get more powerful kernel and consider all 12 months data
* We can also model the solution using Neural Network with multiple layers

---
## References

* Most of the graphical analysis in this notebook is inspired by the excellent Tutorial of Fabien Daniel. https://www.kaggle.com/fabiendaniel/predicting-flight-delays-tutorial
